In [15]:
# %pip install python-dotenv pandas pinecone langchain langchain-openai langchain-pinecone scikit-learn matplotlib


In [16]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL")  # 'gpt-4o-mini'
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_REGION = os.getenv("PINECONE_INDEX_REGION")
PINECONE_INDEX_CLOUD = os.getenv("PINECONE_INDEX_CLOUD")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")  # 'ir'
PINECONE_INDEX_METRIC = os.getenv("PINECONE_INDEX_METRIC")
PINECONE_INDEX_DIMENSION = int(os.getenv("PINECONE_INDEX_DIMENSION"))

# 압축 인덱스 이름
COMPRESSED_INDEX_NAME = f"{PINECONE_INDEX_NAME}-compressed"

print(f"환경 변수 로딩 완료 : {PINECONE_INDEX_NAME}, {PINECONE_INDEX_REGION}, {PINECONE_INDEX_CLOUD}")

환경 변수 로딩 완료 : ir-embeddings, us-east-1, aws


In [17]:
import pandas as pd

# 다중 메타데이터가 포함된 CSV 파일 로딩
documents_df = pd.read_csv("../../datas/documents_meta.csv")
print(f"문서 수: {len(documents_df)}")
documents_df.head()

문서 수: 30


,doc_id,title,content,author,category
0,D1,제주도 여행 가이드,"제주도는 대한민국의 대표 관광지로서, 한라산 등반, 성산 일출봉 관광, 해변 활동(...",김민수,여행
1,D2,전주 비빔밥과 진주 비빔밥 차이점,"비빔밥은 조선 시대부터 전해 내려온 대표적 한국 음식으로, 밥 위에 고명(채소·고기...",이영희,음식
2,D3,걸스데이 히트곡 분석,"걸스데이는 2010년 데뷔한 대한민국의 4인조 걸그룹으로, 대표곡으로는 “Somet...",박지훈,문화;음악
3,D4,세종대왕과 훈민정음,세종대왕(1397~1450)은 훈민정음을 창제하여 한글을 보급한 조선의 4대 임금입...,최수정,역사;교육
4,D5,이순신 장군의 명량 해전,이순신 장군(1545~1598)은 임진왜란 당시 명량 해전에서 13척의 배로 133...,정우성,역사;군사


In [18]:
from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# Pinecone 클라이언트 연결
pc = Pinecone(api_key=PINECONE_API_KEY)

# 인덱스가 없으면 생성
PINECONE_INDEX_NAME = "ir-embeddings-meta"
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=PINECONE_INDEX_DIMENSION,
        metric=PINECONE_INDEX_METRIC,
        spec=ServerlessSpec(region=PINECONE_INDEX_REGION, cloud=PINECONE_INDEX_CLOUD)
    )
index = pc.Index(PINECONE_INDEX_NAME)

# 임베딩 모델 생성
embedding_model = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL, openai_api_key=os.getenv("OPENAI_API_KEY"))
# Pinecone 벡터 스토어 설정
vector_store = PineconeVectorStore(index_name=PINECONE_INDEX_NAME, embedding=embedding_model)

print("Pinecone 및 벡터 스토어 준비 완료")

Pinecone 및 벡터 스토어 준비 완료


In [19]:
from langchain.schema import Document

# Document 리스트 생성
docs_to_upsert = []
for idx, row in documents_df.iterrows():
    # '역사;교육' → ['역사','교육'] 처럼 분리
    cats = row['category'].split(';')
    metadata = {
        'doc_id': row['doc_id'],
        'author': row['author'],
        'category': cats,   # 리스트로 넘기기
    }
    docs_to_upsert.append(Document(page_content=row['content'], metadata=metadata))
    
print(f"업서트할 문서 수: {len(docs_to_upsert)}")

업서트할 문서 수: 30


In [20]:
docs_to_upsert

[Document(metadata={'doc_id': 'D1', 'author': '김민수', 'category': ['여행']}, page_content='제주도는 대한민국의 대표 관광지로서, 한라산 등반, 성산 일출봉 관광, 해변 활동(협재해수욕장·함덕해수욕장) 등이 인기입니다. 현지 음식으로는 흑돼지, 고기국수, 전복죽 등이 있으며, 카페 거리(서귀포시 대정읍 카페 거리)도 유명합니다. 교통은 렌터카나 시외버스를 주로 이용하며, 사전 예약 시 우도 투어나 올레길 트레킹도 즐길 수 있습니다.'),
 Document(metadata={'doc_id': 'D2', 'author': '이영희', 'category': ['음식']}, page_content='비빔밥은 조선 시대부터 전해 내려온 대표적 한국 음식으로, 밥 위에 고명(채소·고기·계란 등)을 올리고 고추장이나 간장을 섞어 먹습니다. 전주 비빔밥은 고명 종류가 다양하고 전주식 고추장을 쓰며, 잔치용으로도 유명합니다. 진주 비빔밥은 고기·회·나물 등을 섞어 더욱 풍부한 식감을 제공합니다. 두 지역 모두 역사적 배경과 재료 구성이 달라 맛과 풍미가 다릅니다.'),
 Document(metadata={'doc_id': 'D3', 'author': '박지훈', 'category': ['문화', '음악']}, page_content='걸스데이는 2010년 데뷔한 대한민국의 4인조 걸그룹으로, 대표곡으로는 “Something”, “Darling”, “Expectation” 등이 있습니다. 데뷔 초기 청순 컨셉에서 점차 섹시·여성미 컨셉으로 변화하며 음원 차트 상위권에 올랐습니다. 멤버 민아·유라·소진·혜리는 드라마·예능·광고 등 다양한 분야에도 진출해 활동 영역을 넓혔습니다.'),
 Document(metadata={'doc_id': 'D4', 'author': '최수정', 'category': ['역사', '교육']}, page_content='세종대왕(1397~1450)은 훈민정음을 창제하여 한글을 보급한 조선

In [21]:
# 벡터 스토어에 다중 메타데이터 문서 업서트
vector_store.add_documents(docs_to_upsert)
print("문서 업서트 완료 — Pinecone 인덱스에 저장됨")

문서 업서트 완료 — Pinecone 인덱스에 저장됨


In [22]:
from langchain.schema import Document

# 메타데이터 필터 예시: category에 '역사'가 포함된 문서만 검색
query = "훈민정음"
results = vector_store.similarity_search(
    query,
    k=5,
    filter={'category': {'$in': ['역사']}}
)
print("필터링된 검색 결과 (category='역사') — 상위 5개 문서 ID:")
for doc in results:
    print(doc.metadata['doc_id'], doc.metadata['category'])

필터링된 검색 결과 (category='역사') — 상위 5개 문서 ID:
D4 ['역사', '교육']
D4 ['역사', '교육']
D15 ['문화', '음악', '역사']
D15 ['문화', '음악', '역사']
D9 ['문화', '음악', '역사']
